# **Network creator**

In [1]:
!netgenerate -g --grid.number 21 --output ../data/large_grid2.net.xml --no-turnarounds

Success.


Manually creating traffic lights...

In [42]:
!python3 $SUMO_HOME/tools/randomTrips.py -n ../data/large_grid2.net.xml -o ../data/flows.xml --begin 0 --end 5000 --flows 500 --jtrrouter --trip-attributes 'departPos="random" departSpeed="max"' --random

In [1]:
import sumolib

In [2]:
net = sumolib.net.readNet("../data/large_grid2.net.xml")

In [51]:
import xml.etree.ElementTree as ET

root = ET.Element("data")
doc = ET.SubElement(root, "interval", begin="0", end="36000")

for i in range(ord('@'), ord('U')): #U not included
    column = chr(i)
    from_edge = column+"10"+chr(i+1)+"10"
    inverse_edge = chr(i+1)+"10"+column+"10"
    top_edge = chr(i+1)+"10"+chr(i+1)+"11"
    bottom_edge = chr(i+1)+"10"+chr(i+1)+"9"
    next_edge = chr(i+1)+"10"+chr(i+2)+"10"
    next_inverse_edge = chr(i+2)+"10"+chr(i+1)+"10"
    #towards
    edge = ET.SubElement(doc, "edgeRelation", to=bottom_edge, probability="0.008")
    edge.set("from", from_edge)
    edge = ET.SubElement(doc, "edgeRelation", to=top_edge, probability="0.002")
    edge.set("from", from_edge)
    edge = ET.SubElement(doc, "edgeRelation", to=next_edge, probability="0.99")
    edge.set("from", from_edge)
    #backwards:
    edge = ET.SubElement(doc, "edgeRelation", to=bottom_edge, probability="0.002")
    edge.set("from", next_inverse_edge)
    edge = ET.SubElement(doc, "edgeRelation", to=top_edge, probability="0.008")
    edge.set("from", next_inverse_edge)
    edge = ET.SubElement(doc, "edgeRelation", to=inverse_edge, probability="0.99")
    edge.set("from", next_inverse_edge)
    
tree = ET.ElementTree(root)
tree.write("turnings.xml")

In [52]:
!jtrrouter -c flow_gen.jtrrcfg --vtype-output vtype.xml

Loading configuration ... done.
Success.up to time step: 4800.00


In [27]:
import numpy as np
import matplotlib.pyplot as plt

from importlib import reload

import tools.mc

P, edge_to_index_map, index_to_edge_map = tools.mc.read_MC("../data/large_grid2_noart.net.xml",
                                                    "../data/turnings2.xml",
                                                    [.3, .5, .2])

In [28]:
def close_edge(P, edge_id):
    P[:, edge_id] = np.zeros(len(P))
    for i in range(len(P)):
        P[i] = P[i]/np.sum(P[i])
    return P

In [29]:
import re

def get_orientation(edge_id):
    #searching the components from ids:
    i = len(edge_id)//2
    while ord(edge_id[i+1])>57: i+=1
    from_, to_ = edge_id[:i], edge_id[i:]
    #edge origin extractor:
    j = 0
    while ord(from_[j])>57: j+=1
    from_col, from_row = from_[:j], int(from_[j:])
    #edge destination extractor:
    j = 0
    while ord(to_[j])>57: j+=1
    to_col, to_row = to_[:j], int(to_[j:])
    orientation = "horizontal" if from_row==to_row else "vertical"
    if orientation=="horizontal":
        direction = "left_to_right" if ord(from_col)<ord(to_col) else "right_to_left"
    else:
        direction = "upward" if from_row<to_row else "downward"
    return orientation, direction, (from_col, from_row), (to_col, to_row)

def invert_edge(edge_id):
        _, _, (f_c, f_r), (t_c, t_r) = get_orientation(edge_id)
        return t_c+str(t_r)+f_c+str(f_r)

In [30]:
def make_col_du_sac(P, edge_index, edge_to_index_map, index_to_edge_map):
    edge = index_to_edge_map[edge_index]
    #no outgoing edges:
    P[edge_index] = np.zeros(len(P))
    inv_edge_index = edge_to_index_map[invert_edge(edge)]
    close_edge(P, inv_edge_index)
    P[edge_index, inv_edge_index] = 1.0
    return P

In [34]:
def change_network(P, edge_to_index_map, index_to_edge_map):
    num_mod = 0
    for i in range(len(P)):
        modification = np.random.choice(["no", "oneway", "coldsac"],1,
                                            p=[.95, .04, .01])[0]
        if modification == "oneway":
            P = close_edge(P, i)
            num_mod += 1
        elif modification == "coldsac":
            P = make_col_du_sac(P, i, edge_to_index_map, index_to_edge_map)
            num_mod += 1
    print(num_mod)
    return P

In [35]:
import xml.etree.ElementTree as ET

def save_turning_probs_to_xml(P, filename):
    root = ET.Element("data")
    doc = ET.SubElement(root, "interval", begin="0", end="36000")
    for i,row in enumerate(P):
        for j,elem in enumerate(row):
            if elem>0.0:
                edge = ET.SubElement(doc, "edgeRelation", to=str(j), probability=str(elem))
                edge.set("from", str(i))
    tree = ET.ElementTree(root)
    tree.write(filename)

In [36]:
P = change_network(P, edge_to_index_map, index_to_edge_map)
save_turning_probs_to_xml(P, "../data/turnings_random.xml")

/home/levente/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


72
